In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', 10)

##### Read in census data for 2012, 2016, and 2020

In [ ]:
df_race_2012 = pd.read_csv('data/election/census/census_race_2012.csv')
df_race_2016 = pd.read_csv('data/election/census/census_race_2016.csv')
df_race_2020 = pd.read_csv('data/election/census/census_race_2020.csv')

In [ ]:
df_age_sex_2012 = pd.read_csv('data/election/census/census_age_sex_2012.csv')
df_age_sex_2016 = pd.read_csv('data/election/census/census_age_sex_2016.csv')
df_age_sex_2020 = pd.read_csv('data/election/census/census_age_sex_2020.csv')

In [ ]:
df_age_employment_2012 = pd.read_csv('data/election/census/census_employment_2012.csv')
df_age_employment_2016 = pd.read_csv('data/election/census/census_employment_2016.csv')
df_age_employment_2020 = pd.read_csv('data/election/census/census_employment_2020.csv')

In [ ]:
df_industry_2012 = pd.read_csv('../../data/election/census/census_industry_2012.csv')
df_industry_2016 = pd.read_csv('../../data/election/census/census_industry_2016.csv')
df_industry_2020 = pd.read_csv('../../data/election/census/census_industry_2020.csv')

In [ ]:
df_housing_financial_2012 = pd.read_csv('data/election/census/census_housing_finanical_2012.csv')
df_housing_financial_2016 = pd.read_csv('data/election/census/census_housing_finanical_2016.csv')
df_housing_financial_2020 = pd.read_csv('data/election/census/census_housing_finanical_2020.csv')

In [ ]:
df_veteran_status_2012 = pd.read_csv('data/election/census/census_veteran_status_2012.csv')
df_veteran_status_2016 = pd.read_csv('data/election/census/census_veteran_status_2016.csv')
df_veteran_status_2020 = pd.read_csv('data/election/census/census_veteran_status_2020.csv')

##### Reshape race data

In [ ]:
import pandas as pd

def reshape_demographics_race_data(df):
    """
    Reshape demographic data from wide format to long format with separate county and state columns.
    
    Parameters:
    df (pandas.DataFrame): Input dataframe with demographic data
    
    Returns:
    pandas.DataFrame: Reshaped dataframe with county and state as rows and demographic categories as columns
    """
    # Reset index to get the Label column as a regular column
    df = df.reset_index()
    
    # Melt the dataframe to convert counties from columns to rows
    melted = pd.melt(
        df,
        id_vars=['Label (Grouping)'],
        var_name='location',
        value_name='value'
    )
    
    # Split the location column into county and state
    # Remove the '!!Estimate' suffix and split on the last comma
    melted['location'] = melted['location'].str.replace('!!Estimate', '')
    melted[['county_name', 'state_name']] = melted['location'].str.rsplit(',', n=1, expand=True)
    
    # Strip whitespace from county and state names
    melted['county_name'] = melted['county_name'].str.strip()
    melted['state_name'] = melted['state_name'].str.strip()
    
    # Pivot the data to get demographic categories as columns
    reshaped = melted.pivot(
        index=['county_name', 'state_name'],
        columns='Label (Grouping)',
        values='value'
    ).reset_index()
    
    # Clean up column names by removing any extra levels
    reshaped.columns.name = None
    
    # Reorder columns to have county_name and state_name first
    demographic_cols = [col for col in reshaped.columns if col not in ['county_name', 'state_name']]
    column_order = ['county_name', 'state_name'] + demographic_cols
    
    return reshaped[column_order]

In [ ]:
df_race_2012_reshaped = reshape_demographics_race_data(df_race_2012)
df_race_2012_reshaped = df_race_2012_reshaped.sort_values(by=['state_name', 'county_name']).reset_index(drop=True)
df_race_2016_reshaped = reshape_demographics_race_data(df_race_2016)
df_race_2016_reshaped = df_race_2016_reshaped.sort_values(by=['state_name', 'county_name']).reset_index(drop=True)
df_race_2020_reshaped = reshape_demographics_race_data(df_race_2020)
df_race_2020_reshaped = df_race_2020_reshaped.sort_values(by=['state_name', 'county_name']).reset_index(drop=True)

In [ ]:
# save the reshaped data to csvs
df_race_2012_reshaped.to_csv('data/election/census/census_race_2012_reshaped.csv', index=False)
df_race_2016_reshaped.to_csv('data/election/census/census_race_2016_reshaped.csv', index=False)
df_race_2020_reshaped.to_csv('data/election/census/census_race_2020_reshaped.csv', index=False)

##### Reshape age data

In [ ]:
import pandas as pd
import numpy as np

def reshape_demographics_age_data(df):
    # First, let's get the column names and split them into components
    columns = df.columns[1:]  # Skip the 'Label (Grouping)' column
    
    # Create a list to store the transformed data
    transformed_data = []
    
    # Track unique counties and states
    processed_locations = set()
    
    # Iterate through each location column
    for col in columns:
        # Split the column name
        parts = col.split('!!')
        
        # Extract location and demographic type
        location = parts[0]
        demo_type = parts[1]  # Male, Female, or Total
        estimate = parts[2] if len(parts) > 2 else None
        
        # Skip if not an Estimate
        if estimate != 'Estimate':
            continue
            
        # Split location into county and state
        county_state = location.split(', ')
        county = county_state[0]
        state = county_state[1]
        
        # Create unique key for this county
        location_key = (county, state)
        
        # If we haven't processed this county yet, create a new row
        if location_key not in processed_locations:
            row_data = {
                'county_name': county,
                'state_name': state
            }
            
            # Add data for each demographic label
            for idx, label in enumerate(df['Label (Grouping)']):
                if pd.notna(df.iloc[idx][col]) and str(df.iloc[idx][col]) != '(X)':
                    value = df.iloc[idx][col]
                    # Convert percentages to floats
                    if isinstance(value, str) and '%' in value:
                        value = float(value.strip('%'))/100
                    
                    # Create column name combining label and demographic type
                    column_name = f"{label}_{demo_type.lower()}"
                    row_data[column_name] = value
            
            transformed_data.append(row_data)
            processed_locations.add(location_key)
        else:
            # Update existing row with additional demographic type data
            row_idx = next(i for i, row in enumerate(transformed_data) 
                          if row['county_name'] == county and row['state_name'] == state)
            
            for idx, label in enumerate(df['Label (Grouping)']):
                if pd.notna(df.iloc[idx][col]) and str(df.iloc[idx][col]) != '(X)':
                    value = df.iloc[idx][col]
                    # Convert percentages to floats
                    if isinstance(value, str) and '%' in value:
                        value = float(value.strip('%'))/100
                    
                    # Create column name combining label and demographic type
                    column_name = f"{label}_{demo_type.lower()}"
                    transformed_data[row_idx][column_name] = value
    
    # Create new dataframe
    new_df = pd.DataFrame(transformed_data)
    
    # Sort columns alphabetically after county_name and state_name
    cols = ['county_name', 'state_name'] + sorted([col for col in new_df.columns 
                                                 if col not in ['county_name', 'state_name']])
    new_df = new_df[cols]
    
    return new_df

# Example usage:
# reshaped_df = reshape_demographics_data(your_df)

In [ ]:
df_age_sex_2012_reshaped = reshape_demographics_age_data(df_age_sex_2012)
df_age_sex_2012_reshaped = df_age_sex_2012_reshaped.sort_values(by=['state_name', 'county_name']).reset_index(drop=True)
df_age_sex_2016_reshaped = reshape_demographics_age_data(df_age_sex_2016)
df_age_sex_2016_reshaped = df_age_sex_2016_reshaped.sort_values(by=['state_name', 'county_name']).reset_index(drop=True)
df_age_sex_2020_reshaped = reshape_demographics_age_data(df_age_sex_2020)
df_age_sex_2020_reshaped = df_age_sex_2020_reshaped.sort_values(by=['state_name', 'county_name']).reset_index(drop=True)

In [ ]:
# save the reshaped data to csvs
df_age_sex_2012_reshaped.to_csv('data/election/census/census_age_sex_2012_reshaped.csv', index=False)
df_age_sex_2016_reshaped.to_csv('data/election/census/census_age_sex_2016_reshaped.csv', index=False)
df_age_sex_2020_reshaped.to_csv('data/election/census/census_age_sex_2020_reshaped.csv', index=False)

##### Reshape Employment data

In [ ]:
import pandas as pd
import numpy as np

def reshape_employment_data(df, year):
    """
    Reshape county-level census data from wide format to long format with counties as rows,
    creating separate columns for Total, Labor Force, Employed, and Unemployment rate for each category.
    
    Parameters:
    df (pandas.DataFrame): Input DataFrame with county data in wide format
    year (int): Census year (2012 or 2016/2020) to determine column naming format
    
    Returns:
    pandas.DataFrame: Reshaped DataFrame with counties as rows
    """
    # Define metric mappings based on year
    if year == 2012:
        metric_mappings = {
            'Total': 'Total',
            'In labor force': 'labor_force',
            'Employed': 'employed',
            'Unemployment rate': 'unemployment_rate'
        }
    else:  # 2016 or 2020
        metric_mappings = {
            'Total': 'Total',
            'Labor Force Participation Rate': 'labor_force',
            'Employment/Population Ratio': 'employed',
            'Unemployment rate': 'unemployment_rate'
        }
    
    # Get the Label column values as our new column names, filtering out header rows
    row_categories = []
    for idx, label in enumerate(df['Label (Grouping)']):
        if pd.notna(label):
            # Skip if row is all caps and contains no data (header row)
            if not (str(label).isupper() and df.iloc[idx, 1:].isna().all()):
                row_categories.append((idx, label))
    
    # Process each unique county
    counties_data = []
    
    # Get unique county-state combinations
    county_columns = [col for col in df.columns if '!!' in col]
    unique_counties = set()
    for col in county_columns:
        county_state = col.split('!!')[0]
        if ',' in county_state:
            unique_counties.add(county_state)
            
    # Process each county
    for county_state in unique_counties:
        county_name, state_name = county_state.split(',', 1)
        state_name = state_name.strip()
        
        # Create a dictionary for this county's data
        county_data = {
            'county_name': county_name,
            'state_name': state_name
        }
        print('processing', county_name, state_name)
        
        # For each demographic category
        for idx, category in row_categories:
            category_clean = (category.replace(' ', '_')
                            .replace('(', '')
                            .replace(')', '')
                            .replace(',', '')
                            .replace('/', '_')
                            .replace('-', '_')
                            .replace('...', ''))
            
            # Get the four metrics for this category using the appropriate mapping
            for original_metric, clean_metric in metric_mappings.items():
                col_name = f"{county_state}!!{original_metric}!!Estimate"
                if col_name in df.columns:
                    value = df.iloc[idx][col_name]
                    
                    # Convert percentage strings to floats if possible
                    if isinstance(value, str):
                        if value in ['-', '(X)']:
                            value = None
                        elif '%' in value:
                            try:
                                value = float(value.strip('%')) / 100
                            except ValueError:
                                pass
                    
                    # Create clean column name
                    final_col_name = f"{category_clean}_{clean_metric}"
                    county_data[final_col_name] = value
        
        counties_data.append(county_data)
    
    # Create the final DataFrame
    result_df = pd.DataFrame(counties_data)
    
    return result_df

In [ ]:
df_age_employment_2012_reshaped = reshape_employment_data(df_age_employment_2012, 2012)
df_age_employment_2012_reshaped = df_age_employment_2012_reshaped.sort_values(by=['state_name', 'county_name']).reset_index(drop=True)

In [ ]:
df_age_employment_2016_reshaped = reshape_employment_data(df_age_employment_2016, 2016)
df_age_employment_2016_reshaped = df_age_employment_2016_reshaped.sort_values(by=['state_name', 'county_name']).reset_index(drop=True)

In [ ]:
df_age_employment_2020_reshaped = reshape_employment_data(df_age_employment_2020, 2020)
df_age_employment_2020_reshaped = df_age_employment_2020_reshaped.sort_values(by=['state_name', 'county_name']).reset_index(drop=True)

In [ ]:
# save the reshaped data to csvs
df_age_employment_2012_reshaped.to_csv('data/election/census/census_age_employment_2012_reshaped.csv', index=False)
df_age_employment_2016_reshaped.to_csv('data/election/census/census_age_employment_2016_reshaped.csv', index=False)
df_age_employment_2020_reshaped.to_csv('data/election/census/census_age_employment_2020_reshaped.csv', index=False)

##### Reshape Industry data

In [ ]:
import pandas as pd
import numpy as np

def reshape_industry_data(df):
    """
    Reshape county-level occupation data from wide format to long format with counties as rows,
    creating separate columns for total workers and percentage in each occupation category.
    
    Parameters:
    df (pandas.DataFrame): Input DataFrame with county occupation data in wide format
    
    Returns:
    pandas.DataFrame: Reshaped DataFrame with counties as rows
    """
    # Get the Label column values as our new column names, filtering out header rows
    row_categories = []
    for idx, label in enumerate(df['Label (Grouping)']):
        if pd.notna(label):
            # Skip if row is all caps or percent imputed rows
            if not (str(label).isupper() or 'PERCENT' in str(label)):
                row_categories.append((idx, label))
    
    # Define occupation categories in order
    occupation_categories = [
        'Total',
        'Management, business, science, and arts occupations',
        'Service occupations',
        'Sales and office occupations',
        'Natural resources, construction, and maintenance occupations',
        'Production, transportation, and material moving occupations'
    ]
    
    # Process each unique county
    counties_data = []
    
    # Get unique county-state combinations
    county_columns = [col for col in df.columns if '!!' in col]
    unique_counties = set()
    for col in county_columns:
        county_state = col.split('!!')[0]
        if ',' in county_state:
            unique_counties.add(county_state)
    
    # Process each county
    for county_state in unique_counties:
        county_name, state_name = county_state.split(',', 1)
        state_name = state_name.strip()
        
        # Create a dictionary for this county's data
        county_data = {
            'county_name': county_name,
            'state_name': state_name
        }
        
        # For each industry category
        for idx, category in row_categories:
            category_clean = (category.replace(' ', '_')
                            .replace('(', '')
                            .replace(')', '')
                            .replace(',', '')
                            .replace('/', '_')
                            .replace('-', '_')
                            .replace('.', '')
                            .replace('...', ''))
            
            # Get values for total and each occupation percentage
            for occupation in occupation_categories:
                col_name = f"{county_state}!!{occupation}!!Estimate"
                if col_name in df.columns:
                    value = df.iloc[idx][col_name]
                    
                    # Convert percentage strings to floats if possible
                    if isinstance(value, str):
                        if value == '-' or value == '(X)':
                            value = None
                        elif '%' in value:
                            try:
                                value = float(value.strip('%')) / 100
                            except ValueError:
                                pass
                    
                    # Create clean column name
                    if occupation == 'Total':
                        suffix = 'total_workers'
                    else:
                        occupation_clean = (occupation.replace(' ', '_')
                                         .replace(',', '')
                                         .replace('_and_', '_')
                                         .lower())
                        suffix = f'pct_{occupation_clean}'
                    
                    final_col_name = f"{category_clean}_{suffix}"
                    county_data[final_col_name] = value
        
        counties_data.append(county_data)
    
    # Create the final DataFrame
    result_df = pd.DataFrame(counties_data)
    
    return result_df

In [ ]:
def get_sorted_common_columns(df_2012, df_2016, df_2020):
    """
    Find common columns across three dataframes and sort them by geography 
    (State, then County within State).
    
    Parameters:
    df_2012, df_2016, df_2020 (pandas.DataFrame): Industry dataframes for different years
    
    Returns:
    tuple: (df_2012_sorted, df_2016_sorted, df_2020_sorted)
    """

    # df_2012_sorted.columns = [col.replace('!!Estimate', '') for col in df_2012_sorted.columns]
    df_2012.columns = [col.replace('!!Estimate', '') for col in df_2012.columns]
    df_2016.columns = [col.replace('!!Estimate', '') for col in df_2016.columns]
    df_2020.columns = [col.replace('!!Estimate', '') for col in df_2020.columns]

    # Find common columns
    common_columns = list(set(df_2012.columns) & set(df_2016.columns) & set(df_2020.columns))

    # # show columns that are not in common_columns
    # print('uncommon 2012:')
    # for col in df_2012.columns:
    #     if col not in common_columns:
    #         print(col)

    # print('uncommon 2016:')
    # for col in df_2016.columns:
    #     if col not in common_columns:
    #         print(col)

    # print('uncommon 2020:')
    # for col in df_2020.columns:
    #     if col not in common_columns:
    #         print(col)
    
    
    # Keep 'Label (Grouping)' as first column if it exists
    if 'Label (Grouping)' in common_columns:
        common_columns.remove('Label (Grouping)')
        sorted_columns = ['Label (Grouping)']
    else:
        sorted_columns = []
    
    # Function to extract state and county from column name
    def get_location_key(col_name):
        if '!!' not in col_name:
            return ('', '', col_name)
        
        location = col_name.split('!!')[0]
        category = col_name.split('!!')[1]
        
        # Handle different location formats
        if ',' in location:
            county, state = location.split(',', 1)
            return (state.strip(), county.strip(), category)
        return (location, '', category)
    
    # Sort the geographical columns
    geo_columns = [col for col in common_columns if '!!' in col]
    sorted_geo_columns = sorted(geo_columns, key=get_location_key)
    
    # Add any remaining non-geographic columns at the end
    remaining_columns = [col for col in common_columns if '!!' not in col and col != 'Label (Grouping)']
    sorted_columns.extend(sorted_geo_columns + remaining_columns)
    
    # Create new dataframes with sorted columns
    df_2012_sorted = df_2012[sorted_columns]
    df_2016_sorted = df_2016[sorted_columns]
    df_2020_sorted = df_2020[sorted_columns]
    
    return df_2012_sorted, df_2016_sorted, df_2020_sorted

# Example usage:
df_2012_sorted, df_2016_sorted, df_2020_sorted = get_sorted_common_columns(
    df_industry_2012,
    df_industry_2016,
    df_industry_2020
)

# Verify the sorting
def verify_sorting(df):
    """Print first few column names to verify sorting"""
    print("First few columns:")
    for col in list(df.columns)[:5]:
        print(f"  {col}")
    print("...")

verify_sorting(df_2012_sorted)

In [ ]:
def standardize_industry_dataframes(df_2012, df_2016, df_2020):
    """
    Standardize Label (Grouping) values across three industry dataframes.
    
    Parameters:
    df_2012, df_2016, df_2020 (pandas.DataFrame): Industry dataframes for different years
    
    Returns:
    tuple: (df_2012_clean, df_2016_clean, df_2020_clean)
    """
    def standardize_label(label):
        # Remove non-breaking space characters and regular spaces
        clean_label = label.replace('\xa0', '').strip()
        
        # Map the percent imputed/allocated variations to a standard form
        if clean_label in ['PERCENT IMPUTED', 'PERCENT ALLOCATED']:
            clean_label = 'PERCENT ALLOCATED'
            
        return clean_label
    
    # Create copies of the original dataframes to avoid modifying them
    df_2012_clean = df_2012.copy()
    df_2016_clean = df_2016.copy()
    df_2020_clean = df_2020.copy()
    
    # Apply standardization to each dataframe
    df_2012_clean['Label (Grouping)'] = df_2012_clean['Label (Grouping)'].apply(standardize_label)
    df_2016_clean['Label (Grouping)'] = df_2016_clean['Label (Grouping)'].apply(standardize_label)
    df_2020_clean['Label (Grouping)'] = df_2020_clean['Label (Grouping)'].apply(standardize_label)
    
    return df_2012_clean, df_2016_clean, df_2020_clean

# Example usage:
df_industry_2012_clean, df_industry_2016_clean, df_industry_2020_clean = standardize_industry_dataframes(
    df_2012_sorted,
    df_2016_sorted,
    df_2020_sorted
)

In [ ]:
# create lists of all Label (Grouping) values
label_grouping_2012 = df_industry_2012_clean['Label (Grouping)'].tolist()
label_grouping_2016 = df_industry_2016_clean['Label (Grouping)'].tolist()
label_grouping_2020 = df_industry_2020_clean['Label (Grouping)'].tolist()

In [ ]:
def analyze_occupation_categories(df):
    """
    Extract unique occupation subcategories from column names and
    verify consistency across all locations.
    
    Parameters:
    df (pandas.DataFrame): Dataframe with occupation data
    
    Returns:
    tuple: (list of unique categories, dict of inconsistencies if any)
    """
    # Get columns that contain occupation data (have '!!' delimiter)
    occupation_columns = [col for col in df.columns if '!!' in col]
    
    # Extract categories and group by location
    location_categories = {}
    all_categories = set()
    
    for col in occupation_columns:
        location, category = col.split('!!', 1)
        all_categories.add(category)
        
        if location not in location_categories:
            location_categories[location] = set()
        location_categories[location].add(category)
    
    # Sort categories for consistent output
    sorted_categories = sorted(list(all_categories))
    
    # Check for inconsistencies
    inconsistencies = {}
    for location, categories in location_categories.items():
        missing = all_categories - categories
        extra = categories - all_categories
        if missing or extra:
            inconsistencies[location] = {
                'missing': sorted(list(missing)),
                'extra': sorted(list(extra))
            }
    
    # Print summary
    print(f"Found {len(sorted_categories)} unique occupation categories:")
    for i, category in enumerate(sorted_categories, 1):
        print(f"{i}. {category}")
    
    print(f"\nChecked {len(location_categories)} locations")
    if inconsistencies:
        print(f"Found inconsistencies in {len(inconsistencies)} locations:")
        for location, issues in inconsistencies.items():
            print(f"\n{location}:")
            if issues['missing']:
                print("  Missing categories:")
                for cat in issues['missing']:
                    print(f"    - {cat}")
            if issues['extra']:
                print("  Extra categories:")
                for cat in issues['extra']:
                    print(f"    - {cat}")
    else:
        print("\nAll locations have consistent categories!")
    
    return sorted_categories, inconsistencies

# Example usage:
categories, inconsistencies = analyze_occupation_categories(df_industry_2012_clean)

# If you just want the list of categories:
occupation_categories = categories

print("\nOccupation categories list:")
print(occupation_categories)

In [ ]:
df_industry_2016_clean

In [ ]:
# drop the row with 'Label (Grouping)' == 'PERCENT ALLOCATED'
df_industry_2012_clean = df_industry_2012_clean[df_industry_2012_clean['Label (Grouping)'] != 'PERCENT ALLOCATED']
df_industry_2016_clean = df_industry_2016_clean[df_industry_2016_clean['Label (Grouping)'] != 'PERCENT ALLOCATED']
df_industry_2020_clean = df_industry_2020_clean[df_industry_2020_clean['Label (Grouping)'] != 'PERCENT ALLOCATED']

In [211]:
def reshape_county_data(df):
    """
    Reshape data to have one row per county with:
    1. OCCUPATION_category_INDUSTRY_label columns for occupation percentages
    2. INDUSTRY_label_total columns for raw counts
    3. INDUSTRY_label_percent columns for industry percentages of total employment
    
    Parameters:
    df (pandas.DataFrame): Input dataframe
    
    Returns:
    pandas.DataFrame: Reshaped dataframe with one row per county
    """
    # Get unique counties
    counties = set()
    for col in df.columns:
        if '!!' in col:
            county = col.split('!!')[0]
            counties.add(county)
    
    # Define occupation categories
    occupations = [
        'Management, business, science, and arts occupations',
        'Natural resources, construction, and maintenance occupations',
        'Production, transportation, and material moving occupations',
        'Sales and office occupations',
        'Service occupations',
        'Total'
    ]
    
    county_data = []
    
    for county in counties:
        row_data = {}
        
        # Add county and state information
        if ',' in county:
            county_name, state = county.split(',', 1)
            row_data['county'] = county_name.strip()
            row_data['state'] = state.strip()
        else:
            row_data['location'] = county
            row_data['state'] = ''
        
        # First pass: Get total employed population for percentage calculations
        total_employed = None
        for idx, row in df.iterrows():
            label = row['Label (Grouping)']
            if pd.isna(label) or label.strip() == '':
                continue
                
            if label == 'Civilian employed population 16 years and over':
                total_col = f"{county}!!Total"
                if total_col in df.columns:
                    value = row[total_col]
                    if value != '(X)':
                        try:
                            total_employed = float(str(value).replace(',', ''))
                            row_data['total_civilian_employed'] = total_employed
                        except ValueError:
                            pass
                break
        
        # Process each industry label
        for idx, row in df.iterrows():
            label = row['Label (Grouping)']
            if pd.isna(label) or label.strip() == '':
                continue
            
            # Special handling for Industry row
            if label == 'Industry':
                total_col = f"{county}!!Total"
                if total_col in df.columns:
                    value = row[total_col]
                    if value != '(X)':
                        row_data['INDUSTRY_industrial_total'] = value
                continue
            
            # Clean industry label
            clean_industry = (label.lower()
                            .replace(' ', '_')
                            .replace(',', '')
                            .replace('...', '')
                            .replace('(', '')
                            .replace(')', '')
                            .replace('-', '_')
                            .strip('_'))
            
            # For all other rows, get all occupation categories
            for occ in occupations:
                col = f"{county}!!{occ}"
                if col in df.columns:
                    value = row[col]
                    if value != '(X)' and not pd.isna(value):
                        # Clean occupation name
                        clean_occ = (occ.lower()
                                   .replace(' ', '_')
                                   .replace(',', '')
                                   .replace('(', '')
                                   .replace(')', '')
                                   .replace('-', '_'))
                        
                        if occ == 'Total':
                            # For total values, use simpler naming
                            col_name = f"INDUSTRY_{clean_industry}_total"
                            # Add the industry percentage if we have total employed
                            if total_employed is not None:
                                try:
                                    total_value = float(str(value).replace(',', ''))
                                    percent_col_name = f"INDUSTRY_{clean_industry}_percent"
                                    row_data[percent_col_name] = total_value / total_employed
                                except (ValueError, ZeroDivisionError):
                                    pass
                        else:
                            # For occupation percentages, use OCCUPATION_category_INDUSTRY_label format
                            col_name = f"INDUSTRY_{clean_industry}_OCCUPATION_{clean_occ}_percent"
                        
                        # Convert percentage strings to floats
                        if isinstance(value, str):
                            if '%' in value:
                                value = float(value.strip('%')) / 100
                            else:
                                try:
                                    value = float(value.replace(',', ''))
                                except ValueError:
                                    pass
                        
                        row_data[col_name] = value
        
        county_data.append(row_data)
    
    # Convert to dataframe
    result_df = pd.DataFrame(county_data)
    
    # Sort columns to group related fields together
    location_cols = ['county', 'state', 'location', 'total_civilian_employed']
    industry_total_cols = [col for col in result_df.columns if col.endswith('_total')]
    # industry_percent_cols = [col for col in result_df.columns if col.endswith('_percent')]
    industry_percent_cols = [col for col in result_df.columns if 'INDUSTRY_' in col and '_percent' in col]
    occupation_cols = [col for col in result_df.columns if col.startswith('OCCUPATION_')]
    
    sorted_columns = (
        [col for col in location_cols if col in result_df.columns] +
        sorted(industry_total_cols) +
        sorted(industry_percent_cols) +
        sorted(occupation_cols)
    )
    
    result_df = result_df[sorted_columns]
    
    return result_df

# Example usage:
reshaped_df = reshape_county_data(df_industry_2012_clean)

# Display info about the transformed data
print(f"\nReshaped dataframe has {len(reshaped_df)} rows (counties) and {len(reshaped_df.columns)} columns")
print("\nExample column names:")
print("\nLocation columns:")
print([col for col in reshaped_df.columns if col in ['county', 'state', 'location']][:5])
print("\nIndustry total columns:")
print([col for col in reshaped_df.columns if col.endswith('_total')][:5])
print("\nOccupation percentage columns:")
print([col for col in reshaped_df.columns if col.startswith('OCCUPATION_')][:5])


Reshaped dataframe has 3216 rows (counties) and 102 columns

Example column names:

Location columns:
['county', 'state']

Industry total columns:
['INDUSTRY_agriculture_forestry_fishing_and_hunting_and_mining_total', 'INDUSTRY_arts_entertainment_and_recreation_and_accommodation_and_food_services_total', 'INDUSTRY_civilian_employed_population_16_years_and_over_total', 'INDUSTRY_construction_total', 'INDUSTRY_educational_services_and_health_care_and_social_assistance_total']

Occupation percentage columns:
[]


In [212]:
# change col name INDUSTRY_industrial_total to INDUSTRY_industrial_percent and convert the value from a string with percnt sign to a float
reshaped_df = reshaped_df.rename(columns={'INDUSTRY_industrial_total': 'INDUSTRY_industrial_percent'})
reshaped_df['INDUSTRY_industrial_percent'] = reshaped_df['INDUSTRY_industrial_percent'].str.replace('%', '').astype(float)/100

In [213]:
# for columns that contain 'civilian_employed_population_16_years_and_over', change that part to 'employed_total'
reshaped_df.columns = [col.replace('civilian_employed_population_16_years_and_over', 'employed') for col in reshaped_df.columns]

In [214]:
pd.set_option('display.max_columns', 200)

In [216]:
df_industry_2012_reshaped = reshape_county_data(df_industry_2012_clean)
df_industry_2016_reshaped = reshape_county_data(df_industry_2016_clean)
df_industry_2020_reshaped = reshape_county_data(df_industry_2020_clean)

df_industry_2012_reshaped = df_industry_2012_reshaped.rename(columns={'INDUSTRY_industrial_total': 'INDUSTRY_industrial_percent'})
df_industry_2012_reshaped['INDUSTRY_industrial_percent'] = df_industry_2012_reshaped['INDUSTRY_industrial_percent'].str.replace('%', '').astype(float)/100
df_industry_2016_reshaped = df_industry_2016_reshaped.rename(columns={'INDUSTRY_industrial_total': 'INDUSTRY_industrial_percent'})
df_industry_2016_reshaped['INDUSTRY_industrial_percent'] = df_industry_2016_reshaped['INDUSTRY_industrial_percent'].str.replace('%', '').astype(float)/100
df_industry_2020_reshaped = df_industry_2020_reshaped.rename(columns={'INDUSTRY_industrial_total': 'INDUSTRY_industrial_percent'})
df_industry_2020_reshaped['INDUSTRY_industrial_percent'] = df_industry_2020_reshaped['INDUSTRY_industrial_percent'].str.replace('%', '').astype(float)/100

df_industry_2012_reshaped.columns = [col.replace('civilian_employed_population_16_years_and_over', 'employed') for col in df_industry_2012_reshaped.columns]
df_industry_2016_reshaped.columns = [col.replace('civilian_employed_population_16_years_and_over', 'employed') for col in df_industry_2016_reshaped.columns]
df_industry_2020_reshaped.columns = [col.replace('civilian_employed_population_16_years_and_over', 'employed') for col in df_industry_2020_reshaped.columns]

df_industry_2012_reshaped.columns = [col.replace('civilian', '') for col in df_industry_2012_reshaped.columns]
df_industry_2016_reshaped.columns = [col.replace('civilian', '') for col in df_industry_2016_reshaped.columns]
df_industry_2020_reshaped.columns = [col.replace('civilian', '') for col in df_industry_2020_reshaped.columns]

df_industry_2012_reshaped = df_industry_2012_reshaped.sort_values(by=['state', 'county']).reset_index(drop=True)
df_industry_2016_reshaped = df_industry_2016_reshaped.sort_values(by=['state', 'county']).reset_index(drop=True)
df_industry_2020_reshaped = df_industry_2020_reshaped.sort_values(by=['state', 'county']).reset_index(drop=True)

In [219]:
# save the reshaped data to csvs
df_industry_2012_reshaped.to_csv('../../data/election/census/census_industry_2012_reshaped.csv', index=False)
df_industry_2016_reshaped.to_csv('../../data/election/census/census_industry_2016_reshaped.csv', index=False)
df_industry_2020_reshaped.to_csv('../../data/election/census/census_industry_2020_reshaped.csv', index=False)

##### Reshape housing financial data

In [ ]:
import pandas as pd
import numpy as np

def reshape_housing_financial_data(df):
    """
    Reshape county-level housing financial data from wide format to long format with counties as rows,
    creating separate columns for total, owner-occupied, and renter-occupied housing metrics.
    
    Parameters:
    df (pandas.DataFrame): Input DataFrame with county housing data in wide format
    
    Returns:
    pandas.DataFrame: Reshaped DataFrame with counties as rows
    """
    # Get the Label column values as our new column names, filtering out header rows
    row_categories = []
    for idx, label in enumerate(df['Label (Grouping)']):
        if pd.notna(label):
            # Skip if row is all caps, header rows, or percent imputed
            if not (str(label).isupper() or 'PERCENT' in str(label)):
                row_categories.append((idx, label))
    
    # Define housing categories
    housing_categories = [
        'Occupied housing units',
        'Owner-occupied housing units',
        'Renter-occupied housing units'
    ]
    
    # Process each unique county
    counties_data = []
    
    # Get unique county-state combinations
    county_columns = [col for col in df.columns if '!!' in col]
    unique_counties = set()
    for col in county_columns:
        county_state = col.split('!!')[0]
        if ',' in county_state:
            unique_counties.add(county_state)
    
    # Process each county
    for county_state in unique_counties:
        county_name, state_name = county_state.split(',', 1)
        state_name = state_name.strip()
        
        # Create a dictionary for this county's data
        county_data = {
            'county_name': county_name,
            'state_name': state_name
        }
        
        # For each financial metric
        for idx, category in row_categories:
            category_clean = (category.replace(' ', '_')
                            .replace('(', '')
                            .replace(')', '')
                            .replace(',', '')
                            .replace('/', '_')
                            .replace('-', '_')
                            .replace('.', '')
                            .replace('...', '')
                            .replace('$', '')
                            .replace('__', '_')
                            .lower())
            
            # Get values for each housing category
            for housing_type in housing_categories:
                col_name = f"{county_state}!!{housing_type}!!Estimate"
                if col_name in df.columns:
                    value = df.iloc[idx][col_name]
                    
                    # Convert percentage strings to floats if possible
                    if isinstance(value, str):
                        if value == '-' or value == '(X)':
                            value = None
                        elif '%' in value:
                            try:
                                value = float(value.strip('%')) / 100
                            except ValueError:
                                pass
                        # Handle dollar amounts
                        elif '$' in value or ',' in value:
                            try:
                                value = float(value.replace('$', '').replace(',', ''))
                            except ValueError:
                                pass
                    
                    # Create clean column name
                    housing_type_clean = (housing_type.replace(' ', '_')
                                        .replace('-', '_')
                                        .lower())
                    final_col_name = f"{category_clean}_{housing_type_clean}"
                    county_data[final_col_name] = value
        
        counties_data.append(county_data)
    
    # Create the final DataFrame
    result_df = pd.DataFrame(counties_data)
    
    return result_df

In [ ]:
df_housing_financial_2012_reshaped = reshape_housing_financial_data(df_housing_financial_2012)
df_housing_financial_2012_reshaped = df_housing_financial_2012_reshaped.sort_values(by=['state_name', 'county_name']).reset_index(drop=True)
df_housing_financial_2016_reshaped = reshape_housing_financial_data(df_housing_financial_2016)
df_housing_financial_2016_reshaped = df_housing_financial_2016_reshaped.sort_values(by=['state_name', 'county_name']).reset_index(drop=True)
df_housing_financial_2020_reshaped = reshape_housing_financial_data(df_housing_financial_2020)
df_housing_financial_2020_reshaped = df_housing_financial_2020_reshaped.sort_values(by=['state_name', 'county_name']).reset_index(drop=True)

In [ ]:
# save the reshaped data to csvs
df_housing_financial_2012_reshaped.to_csv('data/election/census/census_housing_financial_2012_reshaped.csv', index=False)
df_housing_financial_2016_reshaped.to_csv('data/election/census/census_housing_financial_2016_reshaped.csv', index=False)
df_housing_financial_2020_reshaped.to_csv('data/election/census/census_housing_financial_2020_reshaped.csv', index=False)

##### Reshape veteran status data

In [ ]:
import pandas as pd
import numpy as np

def reshape_veteran_data(df):
    """
    Reshape county-level veteran status data from wide format to long format with counties as rows,
    creating separate columns for total population, veterans, and nonveterans metrics.
    
    Parameters:
    df (pandas.DataFrame): Input DataFrame with county veteran data in wide format
    
    Returns:
    pandas.DataFrame: Reshaped DataFrame with counties as rows
    """
    # Get the Label column values as our new column names, filtering out header rows
    row_categories = []
    for idx, label in enumerate(df['Label (Grouping)']):
        if pd.notna(label):
            # Skip if row is all caps, header rows, or percent imputed
            if not (str(label).isupper() or 'PERCENT' in str(label)):
                row_categories.append((idx, label))
    
    # Define status categories
    status_categories = [
        'Total',
        'Veterans',
        'Nonveterans'
    ]
    
    # Process each unique county
    counties_data = []
    
    # Get unique county-state combinations
    county_columns = [col for col in df.columns if '!!' in col]
    unique_counties = set()
    for col in county_columns:
        county_state = col.split('!!')[0]
        if ',' in county_state:
            unique_counties.add(county_state)
    
    # Process each county
    for county_state in unique_counties:
        county_name, state_name = county_state.split(',', 1)
        state_name = state_name.strip()
        
        # Create a dictionary for this county's data
        county_data = {
            'county_name': county_name,
            'state_name': state_name
        }
        
        # For each demographic/service metric
        for idx, category in row_categories:
            category_clean = (category.replace(' ', '_')
                            .replace('(', '')
                            .replace(')', '')
                            .replace(',', '')
                            .replace('/', '_')
                            .replace('-', '_')
                            .replace('.', '')
                            .replace('...', '')
                            .replace('$', '')
                            .replace('__', '_')
                            .lower())
            
            # Special handling for period of service categories
            is_period_of_service = 'veterans' in category_clean and any(period in category_clean 
                                                                      for period in ['gulf_war', 'vietnam', 'korean', 'world_war'])
            
            # Get values for each status category
            for status in status_categories:
                col_name = f"{county_state}!!{status}!!Estimate"
                if col_name in df.columns:
                    value = df.iloc[idx][col_name]
                    
                    # Convert percentage strings to floats if possible
                    if isinstance(value, str):
                        if value == '-' or value == '(X)':
                            value = None
                        elif '%' in value:
                            try:
                                value = float(value.strip('%')) / 100
                            except ValueError:
                                pass
                        # Handle dollar amounts
                        elif '$' in value or ',' in value:
                            try:
                                value = float(value.replace('$', '').replace(',', ''))
                            except ValueError:
                                pass
                    
                    # Create clean column name
                    if is_period_of_service:
                        # For period of service, only store veteran percentage
                        if status == 'Veterans':
                            final_col_name = f"pct_{category_clean}"
                            county_data[final_col_name] = value
                    else:
                        # For other metrics, store all three categories
                        status_clean = status.lower()
                        final_col_name = f"{category_clean}_{status_clean}"
                        county_data[final_col_name] = value
        
        counties_data.append(county_data)
    
    # Create the final DataFrame
    result_df = pd.DataFrame(counties_data)
    
    return result_df

In [ ]:
df_veteran_status_2012_reshaped = reshape_veteran_data(df_veteran_status_2012)
df_veteran_status_2012_reshaped = df_veteran_status_2012_reshaped.sort_values(by=['state_name', 'county_name']).reset_index(drop=True)
df_veteran_status_2016_reshaped = reshape_veteran_data(df_veteran_status_2016)
df_veteran_status_2016_reshaped = df_veteran_status_2016_reshaped.sort_values(by=['state_name', 'county_name']).reset_index(drop=True)
df_veteran_status_2020_reshaped = reshape_veteran_data(df_veteran_status_2020)
df_veteran_status_2020_reshaped = df_veteran_status_2020_reshaped.sort_values(by=['state_name', 'county_name']).reset_index(drop=True)

In [ ]:
# save the reshaped data to csvs
df_veteran_status_2012_reshaped.to_csv('data/election/census/census_veteran_status_2012_reshaped.csv', index=False)
df_veteran_status_2016_reshaped.to_csv('data/election/census/census_veteran_status_2016_reshaped.csv', index=False)
df_veteran_status_2020_reshaped.to_csv('data/election/census/census_veteran_status_2020_reshaped.csv', index=False)

##### Stack dataframes in list for processing

In [221]:
# # race dataframes
# df_race_list = [df_race_2012_reshaped, df_race_2016_reshaped, df_race_2020_reshaped]
# # age_sex dataframes
# df_age_sex_list = [df_age_sex_2012_reshaped, df_age_sex_2016_reshaped, df_age_sex_2020_reshaped]
# # age_employment dataframes
# df_age_employment_list = [df_age_employment_2012_reshaped, df_age_employment_2016_reshaped, df_age_employment_2020_reshaped]
# # industry dataframes
df_industry_list = [df_industry_2012_reshaped, df_industry_2016_reshaped, df_industry_2020_reshaped]
# # housing_financial dataframes
# df_housing_financial_list = [df_housing_financial_2012_reshaped, df_housing_financial_2016_reshaped, df_housing_financial_2020_reshaped]
# # veteran_status dataframes
# df_veteran_status_list = [df_veteran_status_2012_reshaped, df_veteran_status_2016_reshaped, df_veteran_status_2020_reshaped]

##### Fix data

In [ ]:
# drop the row with county_name 'index' in the df_race_list dataframes
for df in df_race_list:
    df.drop(df[df['county_name'] == 'index'].index, inplace=True)
    df.reset_index(drop=True, inplace=True)

In [223]:
# # drop rows where state_name is 'Puerto Rico' or 'Alaska' or 'Hawaii' or 'District of Columbia' in all dataframe lists
# for df in df_race_list:
#     df.drop(df[df['state_name'] == 'Puerto Rico'].index, inplace=True)
#     df.drop(df[df['state_name'] == 'Alaska'].index, inplace=True)
#     df.drop(df[df['state_name'] == 'Hawaii'].index, inplace=True)
#     df.drop(df[df['state_name'] == 'District of Columbia'].index, inplace=True)
#     df.reset_index(drop=True, inplace=True)

# for df in df_age_sex_list:
#     df.drop(df[df['state_name'] == 'Puerto Rico'].index, inplace=True)
#     df.drop(df[df['state_name'] == 'Alaska'].index, inplace=True)
#     df.drop(df[df['state_name'] == 'Hawaii'].index, inplace=True)
#     df.drop(df[df['state_name'] == 'District of Columbia'].index, inplace=True)
#     df.reset_index(drop=True, inplace=True)

# for df in df_age_employment_list:
#     df.drop(df[df['state_name'] == 'Puerto Rico'].index, inplace=True)
#     df.drop(df[df['state_name'] == 'Alaska'].index, inplace=True)
#     df.drop(df[df['state_name'] == 'Hawaii'].index, inplace=True)
#     df.drop(df[df['state_name'] == 'District of Columbia'].index, inplace=True)
#     df.reset_index(drop=True, inplace=True)

for df in df_industry_list:
    df.drop(df[df['state'] == 'Puerto Rico'].index, inplace=True)
    df.drop(df[df['state'] == 'Alaska'].index, inplace=True)
    df.drop(df[df['state'] == 'Hawaii'].index, inplace=True)
    df.drop(df[df['state'] == 'District of Columbia'].index, inplace=True)
    df.reset_index(drop=True, inplace=True)

# for df in df_housing_financial_list:
#     df.drop(df[df['state_name'] == 'Puerto Rico'].index, inplace=True)
#     df.drop(df[df['state_name'] == 'Alaska'].index, inplace=True)
#     df.drop(df[df['state_name'] == 'Hawaii'].index, inplace=True)
#     df.drop(df[df['state_name'] == 'District of Columbia'].index, inplace=True)
#     df.reset_index(drop=True, inplace=True)

# for df in df_veteran_status_list:
#     df.drop(df[df['state_name'] == 'Puerto Rico'].index, inplace=True)
#     df.drop(df[df['state_name'] == 'Alaska'].index, inplace=True)
#     df.drop(df[df['state_name'] == 'Hawaii'].index, inplace=True)
#     df.drop(df[df['state_name'] == 'District of Columbia'].index, inplace=True)
#     df.reset_index(drop=True, inplace=True)

In [ ]:
# show rows where county_name does not contain 'County' or 'Parish' or 'city' or 'Borough
df2[~df2['county_name'].str.contains('County|Parish|city|Borough')]

In [224]:
state_abbr_to_name = {
    'AL': 'Alabama',
    'AZ': 'Arizona',
    'AR': 'Arkansas',
    'CA': 'California',
    'CO': 'Colorado',
    'CT': 'Connecticut',
    'DE': 'Delaware',
    'FL': 'Florida',
    'GA': 'Georgia',
    'ID': 'Idaho',
    'IL': 'Illinois',
    'IN': 'Indiana',
    'IA': 'Iowa',
    'KS': 'Kansas',
    'KY': 'Kentucky',
    'LA': 'Louisiana',
    'ME': 'Maine',
    'MD': 'Maryland',
    'MA': 'Massachusetts',
    'MI': 'Michigan',
    'MN': 'Minnesota',
    'MS': 'Mississippi',
    'MO': 'Missouri',
    'MT': 'Montana',
    'NE': 'Nebraska',
    'NV': 'Nevada',
    'NH': 'New Hampshire',
    'NJ': 'New Jersey',
    'NM': 'New Mexico',
    'NY': 'New York',
    'NC': 'North Carolina',
    'ND': 'North Dakota',
    'OH': 'Ohio',
    'OK': 'Oklahoma',
    'OR': 'Oregon',
    'PA': 'Pennsylvania',
    'RI': 'Rhode Island',
    'SC': 'South Carolina',
    'SD': 'South Dakota',
    'TN': 'Tennessee',
    'TX': 'Texas',
    'UT': 'Utah',
    'VT': 'Vermont',
    'VA': 'Virginia',
    'WA': 'Washington',
    'WV': 'West Virginia',
    'WI': 'Wisconsin',
    'WY': 'Wyoming'
}

state_name_to_abbr = {v: k for k, v in state_abbr_to_name.items()}

In [225]:
# # df2['state_abbr'] = df2['state_name'].map(state_name_to_abbr)

# # for all dataframes lists, add a column for state_abbr
# for df in df_race_list:
#     df['state_abbr'] = df['state_name'].map(state_name_to_abbr)

# for df in df_age_sex_list:
#     df['state_abbr'] = df['state_name'].map(state_name_to_abbr)

# for df in df_age_employment_list:
#     df['state_abbr'] = df['state_name'].map(state_name_to_abbr)

for df in df_industry_list:
    df['state_abbr'] = df['state'].map(state_name_to_abbr)

# for df in df_housing_financial_list:
#     df['state_abbr'] = df['state_name'].map(state_name_to_abbr)

# for df in df_veteran_status_list:
#     df['state_abbr'] = df['state_name'].map(state_name_to_abbr)

In [226]:
# change state to state_name and county to county_name in df_industry_list dataframes
for df in df_industry_list:
    df.rename(columns={'state': 'state_name', 'county': 'county_name'}, inplace=True)

##### Add FIPS codes

In [227]:
import pandas as pd
import unidecode

def clean_place_name(name):
    """Clean place names for consistent matching"""
    if pd.isna(name):
        return name
    
    # Convert to string and clean accented characters
    name = str(name)
    name = unidecode.unidecode(name)
    
    # Standardize spaces and case
    name = name.strip()
    
    # Standardize specific terms
    replacements = {
        'Municipality': 'Municipio',
        'City and Borough': 'Borough',
        'Census Area': 'Census Area',
        ' city': ' City',
        'LaSalle': 'La Salle',
        'LaPorte': 'La Porte',
        'DeSoto': 'De Soto',
        'DeKalb': 'De Kalb',
        'St.': 'St',
        'Ste.': 'Ste'
    }
    
    for old, new in replacements.items():
        name = name.replace(old, new)
    
    return name

def add_fips_codes(census_df, fips_csv_path):
    """
    Add FIPS codes to census data by matching county and state.
    Handles special cases for Alaska, Puerto Rico, and Virginia.
    
    Parameters:
    census_df (pandas.DataFrame): Census dataframe with county_name and state_abbr columns
    fips_csv_path (str): Path to the FIPS CSV file
    
    Returns:
    pandas.DataFrame: Census dataframe with FIPS codes added
    """
    # Read FIPS data
    fips_df = pd.read_csv(fips_csv_path)
    
    # Filter out non-place rows (like "UNITED STATES" and state-level entries)
    fips_df = fips_df[~fips_df['name'].isna()]
    fips_df = fips_df[fips_df['name'].str.contains('County|Borough|Parish|Census Area|city|Municipio', 
                                                  case=False, na=False)]
    
    # Clean up names in both dataframes
    census_df = census_df.copy()
    census_df['clean_name'] = census_df['county_name'].apply(clean_place_name)
    fips_df['clean_name'] = fips_df['name'].apply(clean_place_name)
    
    # # Special handling for DC
    # dc_mask = census_df['county_name'] == 'District of Columbia'
    # census_df.loc[dc_mask, 'state_abbr'] = 'DC'
    
    # # Handle cases where state_abbr might be NaN
    # state_map = {
    #     'Borough': 'AK',
    #     'Census Area': 'AK',
    #     'Municipio': 'PR'
    # }
    
    # for identifier, state_code in state_map.items():
    #     mask = census_df['county_name'].str.contains(identifier, na=False)
    #     census_df.loc[mask, 'state_abbr'] = state_code
    
    # Merge the dataframes
    result = pd.merge(
        census_df,
        fips_df[['fips', 'clean_name', 'state']],
        how='left',
        left_on=['clean_name', 'state_abbr'],
        right_on=['clean_name', 'state']
    )
    
    # Drop the working columns
    result = result.drop(['clean_name', 'state'], axis=1)
    
    # Convert FIPS codes to strings with leading zeros
    result['fips'] = result['fips'].astype(str).str.zfill(5)
    
    # Rename the FIPS column
    result = result.rename(columns={'fips': 'fips_code'})
    
    # Check for unmatched counties
    unmatched = result[result['fips_code'].isna()]
    if len(unmatched) > 0:
        print(f"\nWarning: {len(unmatched)} places could not be matched with FIPS codes:")
        for _, row in unmatched.iterrows():
            print(f"- {row['county_name']}, {row['state_abbr']}")
    
    # Reorder columns to put fips_code after state_name
    cols = list(result.columns)
    state_name_idx = cols.index('state_name')
    cols.remove('fips_code')
    cols.insert(state_name_idx + 1, 'fips_code')
    result = result[cols]
    
    return result

In [ ]:
# # # add fips codes to all dataframes
# # for df in df_race_list:
# #     df = add_fips_codes(df, 'data/election/fips_state_and_county.csv')

# # for df in df_age_sex_list:
# #     df = add_fips_codes(df, 'data/election/fips_state_and_county.csv')

# # for df in df_age_employment_list:
# #     df = add_fips_codes(df, 'data/election/fips_state_and_county.csv')

# # for df in df_industry_list:
# #     df = add_fips_codes(df, 'data/election/fips_state_and_county.csv')

# # for df in df_housing_financial_list:
# #     df = add_fips_codes(df, 'data/election/fips_state_and_county.csv')

# # for df in df_veteran_status_list:
# #     df = add_fips_codes(df, 'data/election/fips_state_and_county.csv')

# df_race_2012_reshaped = add_fips_codes(df_race_2012_reshaped, 'data/election/fips_state_and_county.csv')
# df_race_2016_reshaped = add_fips_codes(df_race_2016_reshaped, 'data/election/fips_state_and_county.csv')
# df_race_2020_reshaped = add_fips_codes(df_race_2020_reshaped, 'data/election/fips_state_and_county.csv')
# df_age_sex_2012_reshaped = add_fips_codes(df_age_sex_2012_reshaped, 'data/election/fips_state_and_county.csv')
# df_age_sex_2016_reshaped = add_fips_codes(df_age_sex_2016_reshaped, 'data/election/fips_state_and_county.csv')
# df_age_sex_2020_reshaped = add_fips_codes(df_age_sex_2020_reshaped, 'data/election/fips_state_and_county.csv')
# df_age_employment_2012_reshaped = add_fips_codes(df_age_employment_2012_reshaped, 'data/election/fips_state_and_county.csv')
# df_age_employment_2016_reshaped = add_fips_codes(df_age_employment_2016_reshaped, 'data/election/fips_state_and_county.csv')
# df_age_employment_2020_reshaped = add_fips_codes(df_age_employment_2020_reshaped, 'data/election/fips_state_and_county.csv')
df_industry_2012_reshaped = add_fips_codes(df_industry_2012_reshaped, '../../data/election/fips_state_and_county.csv')
df_industry_2016_reshaped = add_fips_codes(df_industry_2016_reshaped, '../../data/election/fips_state_and_county.csv')
df_industry_2020_reshaped = add_fips_codes(df_industry_2020_reshaped, '../../data/election/fips_state_and_county.csv')
# df_housing_financial_2012_reshaped = add_fips_codes(df_housing_financial_2012_reshaped, 'data/election/fips_state_and_county.csv')
# df_housing_financial_2016_reshaped = add_fips_codes(df_housing_financial_2016_reshaped, 'data/election/fips_state_and_county.csv')
# df_housing_financial_2020_reshaped = add_fips_codes(df_housing_financial_2020_reshaped, 'data/election/fips_state_and_county.csv')
# df_veteran_status_2012_reshaped = add_fips_codes(df_veteran_status_2012_reshaped, 'data/election/fips_state_and_county.csv')
# df_veteran_status_2016_reshaped = add_fips_codes(df_veteran_status_2016_reshaped, 'data/election/fips_state_and_county.csv')
# df_veteran_status_2020_reshaped = add_fips_codes(df_veteran_status_2020_reshaped, 'data/election/fips_state_and_county.csv')


FileNotFoundError: [Errno 2] No such file or directory: 'data/election/fips_state_and_county.csv'

In [ ]:
pd.set_option('display.max_rows', 200)

In [ ]:
# drop all columns with more than 0 missing values
df_age_employment_2012_reshaped.dropna(axis=1, how='any', inplace=True)
df_age_employment_2016_reshaped.dropna(axis=1, how='any', inplace=True)
df_age_employment_2020_reshaped.dropna(axis=1, how='any', inplace=True)
df_industry_2012_reshaped.dropna(axis=1, how='any', inplace=True)
df_industry_2016_reshaped.dropna(axis=1, how='any', inplace=True)
df_industry_2020_reshaped.dropna(axis=1, how='any', inplace=True)
df_housing_financial_2012_reshaped.dropna(axis=1, how='any', inplace=True)
df_housing_financial_2016_reshaped.dropna(axis=1, how='any', inplace=True)
df_housing_financial_2020_reshaped.dropna(axis=1, how='any', inplace=True)
df_veteran_status_2012_reshaped.dropna(axis=1, how='any', inplace=True)
df_veteran_status_2016_reshaped.dropna(axis=1, how='any', inplace=True)
df_veteran_status_2020_reshaped.dropna(axis=1, how='any', inplace=True)

In [ ]:
# add _2012 to all columns except for county_name, state_name, fips_code, state_abbr
df_race_2012_reshaped.columns = [col + '_2012' if col not in ['county_name', 'state_name', 'fips_code', 'state_abbr'] else col for col in df_race_2012_reshaped.columns]
df_age_sex_2012_reshaped.columns = [col + '_2012' if col not in ['county_name', 'state_name', 'fips_code', 'state_abbr'] else col for col in df_age_sex_2012_reshaped.columns]
df_age_employment_2012_reshaped.columns = [col + '_2012' if col not in ['county_name', 'state_name', 'fips_code', 'state_abbr'] else col for col in df_age_employment_2012_reshaped.columns]
df_industry_2012_reshaped.columns = [col + '_2012' if col not in ['county_name', 'state_name', 'fips_code', 'state_abbr'] else col for col in df_industry_2012_reshaped.columns]
df_housing_financial_2012_reshaped.columns = [col + '_2012' if col not in ['county_name', 'state_name', 'fips_code', 'state_abbr'] else col for col in df_housing_financial_2012_reshaped.columns]
df_veteran_status_2012_reshaped.columns = [col + '_2012' if col not in ['county_name', 'state_name', 'fips_code', 'state_abbr'] else col for col in df_veteran_status_2012_reshaped.columns]

# add _2016 to all columns except for county_name, state_name, fips_code, state_abbr
df_race_2016_reshaped.columns = [col + '_2016' if col not in ['county_name', 'state_name', 'fips_code', 'state_abbr'] else col for col in df_race_2016_reshaped.columns]
df_age_sex_2016_reshaped.columns = [col + '_2016' if col not in ['county_name', 'state_name', 'fips_code', 'state_abbr'] else col for col in df_age_sex_2016_reshaped.columns]
df_age_employment_2016_reshaped.columns = [col + '_2016' if col not in ['county_name', 'state_name', 'fips_code', 'state_abbr'] else col for col in df_age_employment_2016_reshaped.columns]
df_industry_2016_reshaped.columns = [col + '_2016' if col not in ['county_name', 'state_name', 'fips_code', 'state_abbr'] else col for col in df_industry_2016_reshaped.columns]
df_housing_financial_2016_reshaped.columns = [col + '_2016' if col not in ['county_name', 'state_name', 'fips_code', 'state_abbr'] else col for col in df_housing_financial_2016_reshaped.columns]
df_veteran_status_2016_reshaped.columns = [col + '_2016' if col not in ['county_name', 'state_name', 'fips_code', 'state_abbr'] else col for col in df_veteran_status_2016_reshaped.columns]

# add _2020 to all columns except for county_name, state_name, fips_code, state_abbr
df_race_2020_reshaped.columns = [col + '_2020' if col not in ['county_name', 'state_name', 'fips_code', 'state_abbr'] else col for col in df_race_2020_reshaped.columns]
df_age_sex_2020_reshaped.columns = [col + '_2020' if col not in ['county_name', 'state_name', 'fips_code', 'state_abbr'] else col for col in df_age_sex_2020_reshaped.columns]
df_age_employment_2020_reshaped.columns = [col + '_2020' if col not in ['county_name', 'state_name', 'fips_code', 'state_abbr'] else col for col in df_age_employment_2020_reshaped.columns]
df_industry_2020_reshaped.columns = [col + '_2020' if col not in ['county_name', 'state_name', 'fips_code', 'state_abbr'] else col for col in df_industry_2020_reshaped.columns]
df_housing_financial_2020_reshaped.columns = [col + '_2020' if col not in ['county_name', 'state_name', 'fips_code', 'state_abbr'] else col for col in df_housing_financial_2020_reshaped.columns]
df_veteran_status_2020_reshaped.columns = [col + '_2020' if col not in ['county_name', 'state_name', 'fips_code', 'state_abbr'] else col for col in df_veteran_status_2020_reshaped.columns]

In [ ]:
df_race_2020_reshaped

In [ ]:
# merge all dataframes on fips_code
df_race_merged = df_race_2012_reshaped.merge(df_race_2016_reshaped, on='fips_code')
df_race_merged = df_race_merged.merge(df_race_2020_reshaped, on='fips_code')
df_age_sex_merged = df_age_sex_2012_reshaped.merge(df_age_sex_2016_reshaped, on='fips_code')
df_age_sex_merged = df_age_sex_merged.merge(df_age_sex_2020_reshaped, on='fips_code')
df_age_employment_merged = df_age_employment_2012_reshaped.merge(df_age_employment_2016_reshaped, on='fips_code')
df_age_employment_merged = df_age_employment_merged.merge(df_age_employment_2020_reshaped, on='fips_code')
df_industry_merged = df_industry_2012_reshaped.merge(df_industry_2016_reshaped, on='fips_code')
df_industry_merged = df_industry_merged.merge(df_industry_2020_reshaped, on='fips_code')
df_housing_financial_merged = df_housing_financial_2012_reshaped.merge(df_housing_financial_2016_reshaped, on='fips_code')
df_housing_financial_merged = df_housing_financial_merged.merge(df_housing_financial_2020_reshaped, on='fips_code')
df_veteran_status_merged = df_veteran_status_2012_reshaped.merge(df_veteran_status_2016_reshaped, on='fips_code')
df_veteran_status_merged = df_veteran_status_merged.merge(df_veteran_status_2020_reshaped, on='fips_code')

In [ ]:
import geopandas as gpd

# load the existing counties data county_data_with_elections_2012_2016_2020.geojson
counties = gpd.read_file('data/election/final_data/county_data_with_elections_2012_2016_2020.geojson')

In [ ]:
df_veteran_status_merged['fips_code']

In [ ]:
counties['FIPS Code']

In [ ]:
# add a .0 to each fips code in the counties dataframe
counties['FIPS Code'] = counties['FIPS Code'].astype(str) + '.0'

In [ ]:
# merging all data to the counties dataframe
counties_merged = counties.merge(df_race_merged, left_on='FIPS Code', right_on='fips_code')
counties_merged = counties_merged.merge(df_age_sex_merged, left_on='FIPS Code', right_on='fips_code')
counties_merged = counties_merged.merge(df_age_employment_merged, left_on='FIPS Code', right_on='fips_code')
counties_merged = counties_merged.merge(df_industry_merged, left_on='FIPS Code', right_on='fips_code')
counties_merged = counties_merged.merge(df_housing_financial_merged, left_on='FIPS Code', right_on='fips_code')
counties_merged = counties_merged.merge(df_veteran_status_merged, left_on='FIPS Code', right_on='fips_code')

In [ ]:
counties_merged

In [ ]:
# 15 to 19 years_male_2012_y
counties_merged['FIPS Code']

In [ ]:
# show duplicated columns
duplicated_cols = counties_merged.columns[counties_merged.columns.duplicated()]
duplicated_cols

In [ ]:
# drop duplicated columns
counties_merged.drop(columns=duplicated_cols, inplace=True)

In [ ]:
# save counties_merged to a geojson file
counties_merged.to_file('data/election/final_data/county_data_with_elections_2012_2016_2020_census_MAIN.geojson', driver='GeoJSON')

In [ ]:
# split out the dataframes into separate dataframes for each year (2012, 2016, 2020), also take the state_abbr, county_name, state_name, and fips_code columns
counties_merged_2012 = counties_merged[[col for col in counties_merged.columns if '2012' in col] + ['state_abbr', 'county_name', 'state_name', 'FIPS Code', 'geometry']]
counties_merged_2016 = counties_merged[[col for col in counties_merged.columns if '2016' in col] + ['state_abbr', 'county_name', 'state_name', 'FIPS Code', 'geometry']]
# for 2020, include values that contain 2021 as well
counties_merged_2020 = counties_merged[[col for col in counties_merged.columns if '2020' in col or '2021' in col] + ['state_abbr', 'county_name', 'state_name', 'FIPS Code', 'geometry']]

In [ ]:
# show all cols in counties_merged that have 2020 in them
[col for col in counties_merged.columns if '2016' in col]

In [ ]:
counties_merged[[col for col in counties_merged.columns if '2020' in col] + ['state_abbr', 'county_name', 'state_name', 'FIPS Code', 'geometry']]

In [ ]:
# in counties_merged_2012, fix values in all columns that contain a percentage sign in the value (i.e. some columns have values with percentage signs in them, need to be float)
for col in counties_merged_2012.columns:
    if '%' in counties_merged_2012[col].values:
        counties_merged_2012[col] = counties_merged_2012[col].str.replace('%', '').astype(float)

# in counties_merged_2016, fix values in all columns that contain a percentage sign in the value (i.e. some columns have values with percentage signs in them, need to be float)
for col in counties_merged_2016.columns:
    if '%' in counties_merged_2016[col].values:
        counties_merged_2016[col] = counties_merged_2016[col].str.replace('%', '').astype(float)

# in counties_merged_2020, fix values in all columns that contain a percentage sign in the value (i.e. some columns have values with percentage signs in them, need to be float)
for col in counties_merged_2020.columns:
    if '%' in counties_merged_2020[col].values:
        counties_merged_2020[col] = counties_merged_2020[col].str.replace('%', '').astype(float)

In [ ]:
counties_merged_2020.per_point_diff_2020

In [ ]:
# save merged dataframes to geojson files
counties_merged_2012.to_file('data/election/final_data/county_data_with_elections_2012_census_MAIN.geojson', driver='GeoJSON')
counties_merged_2016.to_file('data/election/final_data/county_data_with_elections_2016_census_MAIN.geojson', driver='GeoJSON')
counties_merged_2020.to_file('data/election/final_data/county_data_with_elections_2020_census_MAIN.geojson', driver='GeoJSON')

In [ ]:
counties_merged_2020

In [ ]:
# save the reshaped data to csvs
df_race_2012_reshaped.to_csv('data/election/census/census_race_2012_reshaped.csv', index=False)
df_race_2016_reshaped.to_csv('data/election/census/census_race_2016_reshaped.csv', index=False)
df_race_2020_reshaped.to_csv('data/election/census/census_race_2020_reshaped.csv', index=False)
# save the reshaped data to csvs
df_age_sex_2012_reshaped.to_csv('data/election/census/census_age_sex_2012_reshaped.csv', index=False)
df_age_sex_2016_reshaped.to_csv('data/election/census/census_age_sex_2016_reshaped.csv', index=False)
df_age_sex_2020_reshaped.to_csv('data/election/census/census_age_sex_2020_reshaped.csv', index=False)
# save the reshaped data to csvs
df_age_employment_2012_reshaped.to_csv('data/election/census/census_age_employment_2012_reshaped.csv', index=False)
df_age_employment_2016_reshaped.to_csv('data/election/census/census_age_employment_2016_reshaped.csv', index=False)
df_age_employment_2020_reshaped.to_csv('data/election/census/census_age_employment_2020_reshaped.csv', index=False)
# save the reshaped data to csvs
df_industry_2012_reshaped.to_csv('data/election/census/census_industry_2012_reshaped.csv', index=False)
df_industry_2016_reshaped.to_csv('data/election/census/census_industry_2016_reshaped.csv', index=False)
df_industry_2020_reshaped.to_csv('data/election/census/census_industry_2020_reshaped.csv', index=False)
# save the reshaped data to csvs
df_housing_financial_2012_reshaped.to_csv('data/election/census/census_housing_financial_2012_reshaped.csv', index=False)
df_housing_financial_2016_reshaped.to_csv('data/election/census/census_housing_financial_2016_reshaped.csv', index=False)
df_housing_financial_2020_reshaped.to_csv('data/election/census/census_housing_financial_2020_reshaped.csv', index=False)
# save the reshaped data to csvs
df_veteran_status_2012_reshaped.to_csv('data/election/census/census_veteran_status_2012_reshaped.csv', index=False)
df_veteran_status_2016_reshaped.to_csv('data/election/census/census_veteran_status_2016_reshaped.csv', index=False)
df_veteran_status_2020_reshaped.to_csv('data/election/census/census_veteran_status_2020_reshaped.csv', index=False)

##### Other stuff

In [ ]:
# add _2012 to all column names, remove ':' from column names
df2_fips.columns = [col + '_2012' if col != 'state_name' and col != 'county_name' else col for col in df2_fips.columns]
df2_fips.columns = [col.replace(':', '') for col in df2_fips.columns]
df2_fips

In [ ]:
# remove commas from all values
df2_fips = df2_fips.replace(',', '', regex=True)
df2_fips

In [ ]:
import pandas as pd

def add_demographic_percentages(df):
    """
    Add percentage columns for each demographic category.
    
    Parameters:
    df (pandas.DataFrame): DataFrame with demographic counts
    
    Returns:
    pandas.DataFrame: DataFrame with additional percentage columns
    """
    result = df.copy()
    
    # Define the demographic columns and their new percentage column names
    demographic_cols = {
        'American Indian and Alaska Native alone_2012': 'pct_aian_2012',
        'Asian alone_2012': 'pct_asian_2012',
        'Black or African American alone_2012': 'pct_black_2012',
        'Native Hawaiian and Other Pacific Islander alone_2012': 'pct_nhpi_2012',
        'Some other race alone_2012': 'pct_other_2012',
        'Two or more races_2012': 'pct_two_or_more_2012',
        'White alone_2012': 'pct_white_2012',
        'Two races excluding Some other race, and three or more races_2012': 'pct_two_races_excl_other_2012',
        'Two races including Some other race_2012': 'pct_two_races_incl_other_2012'
    }
    
    # Calculate percentages for each demographic category
    for col, new_col in demographic_cols.items():
        result[new_col] = (result[col] / result['Total_2012'] * 100).round(2)
        
    # Validate that percentages sum to approximately 100
    # Note: We exclude the detailed two races breakdowns as they're subcategories
    main_pct_cols = [
        'pct_aian_2012',
        'pct_asian_2012',
        'pct_black_2012',
        'pct_nhpi_2012',
        'pct_other_2012',
        'pct_two_or_more_2012',
        'pct_white_2012'
    ]
    
    # Calculate sum of percentages
    result['pct_sum'] = result[main_pct_cols].sum(axis=1)
    
    # Check for any significant deviations from 100%
    tolerance = 0.1  # Allow for small rounding differences
    deviations = result[abs(result['pct_sum'] - 100) > tolerance]
    if len(deviations) > 0:
        print(f"\nWarning: {len(deviations)} rows have percentages that don't sum to 100% (±{tolerance}%):")
        for _, row in deviations.iterrows():
            print(f"- {row['county_name']}, {row['state_name']}: {row['pct_sum']:.2f}%")
    
    # Drop the validation column
    result = result.drop('pct_sum', axis=1)
    
    # Group and order columns
    id_cols = ['county_name', 'state_name', 'fips_code_2012', 'state_abbr_2012']
    count_cols = ['Total_2012'] + list(demographic_cols.keys())
    pct_cols = list(demographic_cols.values())
    
    # Reorder columns
    result = result[id_cols + count_cols + pct_cols]
    
    return result

In [ ]:
# Assuming your dataframe is called df
df2_fips = add_demographic_percentages(df2_fips)